In [6]:
import sys
from this import s
import numpy as np

from sklearn.ensemble import GradientBoostingClassifier
from feature_engineering import refuting_features, polarity_features, hand_features, gen_or_load_feats
from feature_engineering import word_overlap_features
from utils.dataset import DataSet
from utils.generate_test_splits import kfold_split, get_stances_for_folds
from utils.score import report_score, LABELS, score_submission

from utils.system import parse_params, check_version
from csv import DictReader
import pandas

import gensim
from gensim.scripts.glove2word2vec import glove2word2vec
from tqdm import tqdm
from nltk import tokenize
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional, Flatten



In [23]:
MAX_SENT_LEN = 50
MAX_VOCAB_SIZE = 40000
EMBEDDING_DIM = 100
BATCH_SIZE = 200
N_EPOCHS = 10
REGULARIZER_HYPERPARAM = 0.01

hyperparam = {
    'batch_size': 200,
    'max_vocab_size': 20000,
    'max_length': 44,
    'embedding_dim': 100,
    'dropout_rate': 0.3,
    'learning_rate': 0.1,
    'n_epochs': 10,
}


def generate_features(stances,dataset,name):
    h, b, y = [],[],[]

    for stance in stances:
        y.append(LABELS.index(stance['Stance']))
        h.append(stance['Headline'])
        b.append(dataset.articles[stance['Body ID']])

    X_overlap = gen_or_load_feats(word_overlap_features, h, b, "features/overlap."+name+".npy")
    X_refuting = gen_or_load_feats(refuting_features, h, b, "features/refuting."+name+".npy")
    X_polarity = gen_or_load_feats(polarity_features, h, b, "features/polarity."+name+".npy")
    X_hand = gen_or_load_feats(hand_features, h, b, "features/hand."+name+".npy")

    X = np.c_[X_hand, X_polarity, X_refuting, X_overlap]
    return X,y

def formatData(tokenizer, x):
    word_seq = [text_to_word_sequence(sent) for sent in x]
    X = tokenizer.texts_to_sequences([' '.join(seq[:MAX_SENT_LEN]) for seq in word_seq])
    X = pad_sequences(X, maxlen=MAX_SENT_LEN, padding='post', truncating='post')
    return X

In [3]:
# check_version()
# parse_params()

#Load the training dataset and generate folds
d = DataSet()
folds,hold_out = kfold_split(d,n_folds=10)
fold_stances, hold_out_stances = get_stances_for_folds(d,folds,hold_out)

# Load the competition dataset
competition_dataset = DataSet("competition_test")
X_competition, y_competition = generate_features(competition_dataset.stances, competition_dataset, "competition")

h, b = [], []
for stance in competition_dataset.stances:
    h.append(stance['Headline'])
    b.append(stance['Body ID'])

answers = {'Headline': h, 'Body ID': b, 'Stance': []}

Xs = dict()
ys = dict()

# Load/Precompute all features now
X_holdout,y_holdout = generate_features(hold_out_stances,d,"holdout")
for fold in fold_stances:
    Xs[fold],ys[fold] = generate_features(fold_stances[fold],d,str(fold))


best_score = 0
best_fold = None

Reading dataset
Total stances: 49972
Total bodies: 1683
Reading dataset
Total stances: 25413
Total bodies: 904


In [4]:
articles = d.articles.values()
sentences = []
for article in articles:
    sentences += tokenize.sent_tokenize(article)

In [7]:
# build tokenizer
word_seq = [text_to_word_sequence(sent) for sent in sentences]
token = Tokenizer(num_words=hyperparam['max_vocab_size'])
token.fit_on_texts([' '.join(seq[:hyperparam['max_length']]) for seq in word_seq])

In [8]:
#build glove embedding vector
embedding_vector = {}
f = open('./glove/glove.6B.100d.txt')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

400001it [00:30, 13094.16it/s]


In [10]:
vocab_size = len(token.word_index.items()) + 1

embedding_matrix = np.zeros((vocab_size,100))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 27235/27235 [00:00<00:00, 81218.21it/s]


In [ ]:
model = Sequential()
model.add(Embedding(
    input_dim=len(embedding_matrix),
    output_dim=hyperparam['embedding_dim'],
    weights=[embedding_matrix],
    input_length=hyperparam['max_length'],
    trainable = False
))
model.add(Bidirectional(LSTM(75, return_sequences=False, name='Bidrectional_lstm_layer1')))
model.add(Flatten())
model.add(Dense(32,activation = 'relu'))
model.add(Dropout(rate=0.8, name='dropout_1')) # Can try varying dropout rates, in paper suggest 0.2
# model.add(Dense(1,activation = 'sigmoid'))
model.add(Dense(4,activation='softmax'))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics = ['accuracy'])

In [ ]:
# Classifier for each fold
for fold in fold_stances:
    ids = list(range(len(folds)))
    del ids[fold]

    X_train = np.vstack(tuple([Xs[i] for i in ids]))
    y_train = np.hstack(tuple([ys[i] for i in ids]))

    X_test = Xs[fold]
    y_test = ys[fold]
    
    X_val = np.array(X_test[:(len(X_test) // 2)])
    y_val = np.array(y_test[:(len(X_test) // 2)])
    x_test = X_test[(len(X_test) // 2):]
    y_test = y_test[(len(X_test) // 2):]
    
#     new model
    model.fit(X_train, y_train,
              batch_size=hyperparam['batch_size'],
              epochs=10,
              validation_data=(X_val, y_val),
              verbose=1)
#     clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)
#     clf.fit(X_train, y_train)

Epoch 1/10
182/182 [==============================] - 64s 349ms/step - loss: 0.3450 - accuracy: 0.8774 - val_loss: 0.3572 - val_accuracy: 0.8550
Epoch 2/10
182/182 [==============================] - 62s 342ms/step - loss: 0.3421 - accuracy: 0.8777 - val_loss: 0.3616 - val_accuracy: 0.8545
Epoch 3/10
182/182 [==============================] - 60s 329ms/step - loss: 0.3415 - accuracy: 0.8785 - val_loss: 0.3736 - val_accuracy: 0.8516
Epoch 4/10
182/182 [==============================] - 50s 274ms/step - loss: 0.3393 - accuracy: 0.8781 - val_loss: 0.3474 - val_accuracy: 0.8584
Epoch 5/10
182/182 [==============================] - 51s 280ms/step - loss: 0.3370 - accuracy: 0.8792 - val_loss: 0.3435 - val_accuracy: 0.8560
Epoch 6/10
182/182 [==============================] - 48s 265ms/step - loss: 0.3373 - accuracy: 0.8790 - val_loss: 0.3480 - val_accuracy: 0.8589
Epoch 7/10
182/182 [==============================] - 56s 307ms/step - loss: 0.3340 - accuracy: 0.8801 - val_loss: 0.3596 - val_ac

In [50]:
predicted = [LABELS[np.argmax(a, axis = 0)] for a in model.predict(x_test)]
actual = [LABELS[int(a)] for a in y_test]

fold_score, _ = score_submission(actual, predicted)
max_fold_score, _ = score_submission(actual, actual)

score = fold_score/max_fold_score

print("Score for fold "+ str(fold) + " was - " + str(score))
if score > best_score:
    best_score = score
    best_fold = model

Score for fold 6 was - 0.773792305467168


In [49]:
#Run on Holdout set and report the final score on the holdout set
predicted = [LABELS[np.argmax(a, axis = 0)] for a in best_fold.predict(X_holdout)]
actual = [LABELS[int(a)] for a in y_holdout]

print("Scores on the dev set")
report_score(actual,predicted)

#Run on competition dataset
predicted = [LABELS[np.argmax(a, axis = 0)] for a in best_fold.predict(X_competition)]
answers["Stance"] = predicted
answers = pandas.DataFrame(answers)
answers.to_csv('answer.csv', index=False, encoding='utf-8')
actual = [LABELS[int(a)] for a in y_competition]
print("Scores on the test set")
report_score(actual,predicted)

Scores on the dev set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    63     |     0     |    614    |    85     |
-------------------------------------------------------------
| disagree  |     9     |     0     |    140    |    13     |
-------------------------------------------------------------
|  discuss  |    33     |     0     |   1584    |    183    |
-------------------------------------------------------------
| unrelated |     7     |     0     |    155    |   6736    |
-------------------------------------------------------------
Score: 3530.0 out of 4448.5	(79.35259076093065%)
Scores on the test set
-------------------------------------------------------------
|           |   agree   | disagree  |  discuss  | unrelated |
-------------------------------------------------------------
|   agree   |    100    |     0     | 

74.84390086900547